In [1]:
# ============================================
# Problem 3(a) - CIFAR-10 Fully Connected NN
# Mohammadmilad_Sayyad
# ============================================

import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# ------------------------
# 1. Device configuration
# ------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ------------------------
# 2. Hyperparameters
# ------------------------
input_dim   = 32 * 32 * 3   # CIFAR-10 images: 3 x 32 x 32
hidden_dim  = 512           # as required
num_classes = 10
batch_size  = 128
num_epochs  = 30            # you can increase/decrease if you want
learning_rate = 0.001

# ------------------------
# 3. CIFAR-10 Dataset
# ------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

print("Train samples:", len(train_dataset))
print("Test samples :", len(test_dataset))

# ------------------------
# 4. Fully Connected Model
# ------------------------
class FCNetOneHidden(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(FCNetOneHidden, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: (batch_size, 3, 32, 32)
        x = x.view(x.size(0), -1)  # flatten to (batch_size, 3072)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)       # logits
        return out

model = FCNetOneHidden(input_dim, hidden_dim, num_classes).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ------------------------
# 5. Training & Evaluation
# ------------------------
def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100.0 * correct / total
    return accuracy

train_losses = []
test_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    model.train()
    start_time = time.time()

    running_loss = 0.0
    total_batches = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        total_batches += 1

    # Average training loss
    avg_train_loss = running_loss / total_batches
    train_losses.append(avg_train_loss)

    # Evaluation accuracy on test set
    test_acc = evaluate(model, test_loader, device)
    test_accuracies.append(test_acc)

    # Epoch time
    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)

    print(f"Epoch [{epoch+1:02d}/{num_epochs}] | "
          f"Time: {epoch_time:.2f}s | "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Test Accuracy: {test_acc:.2f}%")

print("\nTraining finished.")

# Optionally, you can save train_losses, test_accuracies, and epoch_times
# to a file or plot them for your report.


Using device: cuda


100%|██████████| 170M/170M [00:16<00:00, 10.1MB/s]


Train samples: 50000
Test samples : 10000
FCNetOneHidden(
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
Epoch [01/30] | Time: 15.06s | Train Loss: 1.7003 | Test Accuracy: 45.58%
Epoch [02/30] | Time: 13.19s | Train Loss: 1.4675 | Test Accuracy: 47.71%
Epoch [03/30] | Time: 13.27s | Train Loss: 1.3872 | Test Accuracy: 48.66%
Epoch [04/30] | Time: 13.19s | Train Loss: 1.3129 | Test Accuracy: 49.69%
Epoch [05/30] | Time: 13.19s | Train Loss: 1.2582 | Test Accuracy: 49.30%
Epoch [06/30] | Time: 13.02s | Train Loss: 1.2046 | Test Accuracy: 51.11%
Epoch [07/30] | Time: 13.15s | Train Loss: 1.1548 | Test Accuracy: 50.04%
Epoch [08/30] | Time: 13.05s | Train Loss: 1.1078 | Test Accuracy: 51.02%
Epoch [09/30] | Time: 13.42s | Train Loss: 1.0651 | Test Accuracy: 50.64%
Epoch [10/30] | Time: 13.50s | Train Loss: 1.0019 | Test Accuracy: 51.90%
Epoch [11/30] | Time: 14.02s | Train Loss: 0.9683 | Test Ac